In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/OneDrive-JohnsHopkins/000Projects/0000-Infrastructure/0000-RecFld/FieldNN


# PreCode

In [2]:
import os
from fieldnn.dataset import RFGDataset, my_collate_fn
from torch.utils.data import DataLoader

Tensor_folder = 'data/ProcData/FldGrnTensor/'
recfldgrn_list = ['P-InfoGrn',  'P-EC-PNSect-TknzGrn']
full_recfldgrn_list = ['B-' + i for i in recfldgrn_list]

# from the get_grain_fn to get the Elig_Set.
Elig_Set = ['P4', 'P5', 'P6', 'P7']

dataset = RFGDataset(Tensor_folder, recfldgrn_list, Elig_Set, RecRootID = 'PID')
print(len(dataset), '<---- dataset')
dataloader = DataLoader(dataset, batch_size = 4, shuffle = True, collate_fn = my_collate_fn)
print(len(dataloader), '<---- dataset')


for idx, batch in enumerate(dataloader):
    # print(f'\n------ {idx}')
    batch_rfg, batch_y = batch
    for k, v in batch_rfg.items(): print(k, v.shape)
    break
    # for k, v in batch_rfg.items(): print(k, v.shape)
    # print(batch_y.shape)

4 <---- dataset
1 <---- dataset
B-P-InfoGrn_wgt torch.Size([4, 43])
B-P-InfoGrn_tknidx torch.Size([4, 43])
B-P-InfoGrn_fldidx torch.Size([4, 43])
B-P-EC-PNSect-TknzGrn_wgt torch.Size([4, 23, 14, 221])
B-P-EC-PNSect-TknzGrn_tknidx torch.Size([4, 23, 14, 221])
B-P-EC-PNSect-TknzGrn_fldidx torch.Size([4, 23, 14, 221])


In [3]:
# Prepare the Input
RECFLD_TO_TENSOR = {}
for full_recfldgrn in full_recfldgrn_list:
    RECFLD_TO_TENSOR[full_recfldgrn] = {k: v for k, v in batch_rfg.items() if full_recfldgrn in k}
    
[i for i in RECFLD_TO_TENSOR]

['B-P-InfoGrn', 'B-P-EC-PNSect-TknzGrn']

In [4]:
from fieldnn.dataflowfn.embedflowfn import get_EmbeddingBlock_SubUnit
from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List

############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################


default_SubUnitName = 'E'
fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 


df_SubUnit = get_EmbeddingBlock_SubUnit(full_recfldgrn_list, default_SubUnitName)

s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)
df_SubUnit['SubUnit_BasicNN_List'] = s
s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s



s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,E,[B-P-InfoGrn],B-P-Info,3,[expander-CateEmbed],"[{'full_recfldgrn': 'B-P-InfoGrn', 'Info': ['P...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
1,E,[B-P-EC-PNSect-TknzGrn],B-P-EC-PNSect-Tknz,5,[expander-LLMEmbed],"[{'full_recfldgrn': 'B-P-EC-PNSect-TknzGrn', '...","[{'nn_type_nn_name': 'expander-LLMEmbed', 'Bas..."


In [5]:
from fieldnn.module.embedblock import EmbedBlockLayer

EmbedBlock = EmbedBlockLayer(df_SubUnit)

RECFLD_TO_EMBEDTESNOR = EmbedBlock(RECFLD_TO_TENSOR)

for k, v in RECFLD_TO_EMBEDTESNOR.items():
    print(k, v['info'].shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


B-P-Info torch.Size([4, 43, 128])
B-P-EC-PNSect-Tknz torch.Size([4, 23, 14, 221, 128])


# Get Input Tensor

In [6]:
for full_recfld, info_dict in RECFLD_TO_EMBEDTESNOR.items():
    print(full_recfld, info_dict['info'].shape)

B-P-Info torch.Size([4, 43, 128])
B-P-EC-PNSect-Tknz torch.Size([4, 23, 14, 221, 128])


In [7]:
max_layer = max([len(i.split('-')) for i in RECFLD_TO_EMBEDTESNOR])
max_layer

5

# Get df_SubUnit 

In [8]:
from fieldnn.dataflowfn.reprflowfn import get_Repr_dataflow_table
from fieldnn.dataflowfn.reprflowfn import update_df_Repr_dataflow
from fieldnn.dataflowfn.reprflowfn import update_df_Repr_dataflow_completename
from fieldnn.dataflowfn.reprflowfn import get_Repr_SubUnit_List

In [9]:
full_recfldgrn_list = [i for i in RECFLD_TO_EMBEDTESNOR]
full_recfldgrn_list

['B-P-Info', 'B-P-EC-PNSect-Tknz']

In [10]:
df_dataflow = get_Repr_dataflow_table(full_recfldgrn_list)
# df_dataflow

df_dataflow_new = update_df_Repr_dataflow(df_dataflow)
df_dataflow_new

# df_dataflow_new = update_df_Repr_dataflow(df_dataflow, style = 'Reducer&Merger')
# df_dataflow_new

,5,4,3,2
recfldgrn,,,,
P-Info,NaN,NaN,B-P-Info,NaN
PNSect-Tknz,B-P-EC-PNSect-Tknz,B-P-EC-PNSect,B-P-EC,NaN
(Merge)B-P-Info&EC,NaN,NaN,B-P-Info&EC,B-P


In [11]:
df_dataflow = df_dataflow_new.copy()

#######################
default_R_subunit_name = 'R'
default_MR_subunit_name = 'M'
#######################
df_SubUnit = get_Repr_SubUnit_List(df_dataflow, default_R_subunit_name, default_MR_subunit_name)
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid
0,R,[B-P-EC-PNSect-Tknz],B-P-EC-PNSect,5,4
1,R,[B-P-EC-PNSect],B-P-EC,4,3
2,M,"[B-P-Info, B-P-EC]",B-P-Info&EC,3,3
3,R,[B-P-Info&EC],B-P,3,2


# get df_SubUnit Extra Info

## get SubUnit Name's NN List


In [12]:
from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList

In [13]:
############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'ReduceMax',
    'merger': 'MergeConcat',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)


df_SubUnit['SubUnit_BasicNN_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid,SubUnit_BasicNN_List
0,R,[B-P-EC-PNSect-Tknz],B-P-EC-PNSect,5,4,[reducer-ReduceMax]
1,R,[B-P-EC-PNSect],B-P-EC,4,3,[reducer-ReduceMax]
2,M,"[B-P-Info, B-P-EC]",B-P-Info&EC,3,3,[merger-MergeConcat]
3,R,[B-P-Info&EC],B-P,3,2,[reducer-ReduceMax]


## Get Default BasicNN Config List

In [14]:
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List
# get_SubUnit_Default_NNPara_List(SubUnit_BasicNN_List, SubUnit_input_names, fldgrn_folder, learner_default_dict)

In [15]:
SubUnit_info = df_SubUnit.iloc[0]
SubUnit_BasicNN_List = SubUnit_info['SubUnit_BasicNN_List']
SubUnit_input_names = SubUnit_info['input_names']
SubUnit_output_name = SubUnit_info['output_name']
fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 

print(SubUnit_BasicNN_List)

['reducer-ReduceMax']


In [16]:
SubUnit_DefaultBasicNN_List = get_SubUnit_Default_NNPara_List(SubUnit_BasicNN_List, SubUnit_input_names, 
                                                              fldgrn_folder, learner_default_dict)

SubUnit_DefaultBasicNN_List

[{}]

In [17]:

fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 


s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s
df_SubUnit


,SubUnitName,input_names,output_name,input_layerid,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List
0,R,[B-P-EC-PNSect-Tknz],B-P-EC-PNSect,5,4,[reducer-ReduceMax],[{}]
1,R,[B-P-EC-PNSect],B-P-EC,4,3,[reducer-ReduceMax],[{}]
2,M,"[B-P-Info, B-P-EC]",B-P-Info&EC,3,3,[merger-MergeConcat],[{}]
3,R,[B-P-Info&EC],B-P,3,2,[reducer-ReduceMax],[{}]


## Get BasicNN Config List

In [18]:

from fieldnn.configfn.expanderfn import get_expander_para
from fieldnn.configfn.mergerfn import get_merger_para
from fieldnn.configfn.reducerfn import get_reducer_para
from fieldnn.configfn.learnerfn import get_learner_para



def generate_BasicNN_Config(nn_type_nn_name, 
                            input_names_nnlvl, 
                            default_nnpara, 
                            embed_size, 
                            process):
    '''
        please notince here, this function is not the final version yet.
    '''
    nn_type, nn_name = nn_type_nn_name.split('-')

    if nn_type == 'expander':
        
        assert len(input_names_nnlvl) == 1
        # fld = input_names_nnlvl[0].split('-')[-1]
        # Get the output_name_nnlvl
        output_name_nnlvl = input_names_nnlvl[0].split('Grn')[0]
        
        # Get the input_size and output_size
        input_size = None
        output_size = embed_size 
        
        # Get the postprocess
        postprocess = process
        
        # Derive the para
        full_recfldgrn = default_nnpara['full_recfldgrn']
        Info = default_nnpara['Info']
        # para = get_expander_para(nn_name, default_nnpara, embed_size, vocab_tokenizer, init, postprocess)
        para = get_expander_para(full_recfldgrn, Info, embed_size, postprocess)

    elif nn_type == 'reducer': 
        
        assert len(input_names_nnlvl) == 1
        fld = input_names_nnlvl[0].split('-')[-1]
        # Get the output_name_nnlvl
        output_name_nnlvl = input_names_nnlvl[0].replace('-' + fld, '')

        # Get the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size
        output_size = embed_size 
        
        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_reducer_para(nn_name, default_nnpara, input_size, output_size, postprocess)  
        
    elif nn_type == 'merger':
        # generate the output name
        
        assert len(input_names_nnlvl) > 1
        # input_names_nnlvl: ['B-P-EC-A1CDT', 'B-P-EC-A1CV']
        
        childflds = [i.split('-')[-1] for i in input_names_nnlvl]
        # childflds: ['A1CDT', 'A1CV']
        
        fld_childflds = '&'.join([i for i in childflds])
        # fld_childflds: A1CDT&A1CV
        
        # output_prefix = input_names_nnlvl[0].replace('@' + childflds[0].split('@')[-1], '')
        output_prefix = '-'.join(input_names_nnlvl[0].split('-')[:-1])
        
        # output_prefix: B-P-EC-A1C
        output_name_nnlvl = output_prefix + '-' + fld_childflds
        # output_name_nnlvl: B-P-EC-A1C-DT&V
        
        # Prepare the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size
        output_size = embed_size

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_merger_para(nn_name, default_nnpara, input_size, output_size, postprocess) 
        
        
    elif nn_type == 'learner':
        # generate the output name
        assert len(input_names_nnlvl) == 1
        output_name_nnlvl = input_names_nnlvl[0] # just the same name as before
        
        # Prepare the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size
        output_size = embed_size

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_learner_para(nn_name, default_nnpara, input_size, output_size, postprocess)
        
        
    else:
        raise ValueError(f'nn_type {nn_type} is not available yet')

    Basic_Config = {'input_names_nnlvl': input_names_nnlvl, 
                    'output_name_nnlvl': output_name_nnlvl, 
                    f'{nn_type}_para': para}
    
    return Basic_Config


In [19]:

def get_SubUnit_BasicNN_Config_List(SubUnit_BasicNN_List, 
                                    SubUnit_DefaultBasicNN_List, 
                                    SubUnit_input_names, 
                                    SubUnit_output_name, 
                                    embed_size, 
                                    process, 
                                   ):
    
    
    # TODO: also add the layer_idx in order to deal with the learn_layer_para.
    # This function also needs to be updated. 
    
    SubUnit_BasicNN_Config_List = []
    
    # print('\n\n************** SubUnit_BasicNN_List ****************')
    # print(SubUnit_BasicNN_List)
    # print(SubUnit_input_names)
    # print(SubUnit_output_name)
    for basic_nn_idx, nn_type_nn_name in enumerate(SubUnit_BasicNN_List):

        # Get the input_names_nnlvl
        if basic_nn_idx == 0:
            input_names_nnlvl = SubUnit_input_names # this assigments only works for the first iteration
        else:
            input_names_nnlvl = [output_name_nnlvl]
        
        ##############
        default_nnpara = SubUnit_DefaultBasicNN_List[basic_nn_idx] 
        ##############
        
        
        Basic_Config = generate_BasicNN_Config(nn_type_nn_name, 
                                               input_names_nnlvl, 
                                               default_nnpara, 
                                               embed_size, 
                                               process)
        output_name_nnlvl = Basic_Config['output_name_nnlvl']
        BasicNN_Config = {'nn_type_nn_name': nn_type_nn_name, 'Basic_Config': Basic_Config}
        SubUnit_BasicNN_Config_List.append(BasicNN_Config)
        
        # print('==========================')
        # print(basic_nn_idx, nn_type_nn_name)
        # print(input_names_nnlvl, '<-------- input_names_nnlvl')
        # print(output_name_nnlvl, '<-------- output_name_nnlvl')
        
        
        # also check the input_size of dim and output_size of dim

    final_output_name_nnlvl = output_name_nnlvl

    # if not SubUnit_output_name in final_output_name_nnlvl:
    #     print('xxx errors xxx')
    #     print(final_output_name_nnlvl, '<------- final_output_name_nnlvl')
    #     print(SubUnit_output_name, '<------- SubUnit_output_name')
    #     print('xxx errors xxx')
    assert SubUnit_output_name in final_output_name_nnlvl
    # print('\n************** End SubUnit_BasicNN_List ****************')
    return SubUnit_BasicNN_Config_List


In [20]:
############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################

s = get_SubUnit_BasicNN_Config_List(SubUnit_BasicNN_List, 
                                    SubUnit_DefaultBasicNN_List, 
                                    SubUnit_input_names, 
                                    SubUnit_output_name, 
                                    embed_size, 
                                    process, 
                                   )

SubUnit_BasicNN_Config_List = s

In [21]:
SubUnit_BasicNN_Config_List

[{'nn_type_nn_name': 'reducer-ReduceMax',
  'Basic_Config': {'input_names_nnlvl': ['B-P-EC-PNSect-Tknz'],
   'output_name_nnlvl': 'B-P-EC-PNSect',
   'reducer_para': {'nn_type': 'reducer',
    'nn_name': 'ReduceMax',
    'nn_para': {},
    'input_size': 128,
    'output_size': 128,
    'postprocess': {'activator': 'gelu',
     'dropout': {'p': 0.5, 'inplace': False},
     'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}}}}]

In [22]:
############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################

In [23]:
s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

s

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit


,SubUnitName,input_names,output_name,input_layerid,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,R,[B-P-EC-PNSect-Tknz],B-P-EC-PNSect,5,4,[reducer-ReduceMax],[{}],"[{'nn_type_nn_name': 'reducer-ReduceMax', 'Bas..."
1,R,[B-P-EC-PNSect],B-P-EC,4,3,[reducer-ReduceMax],[{}],"[{'nn_type_nn_name': 'reducer-ReduceMax', 'Bas..."
2,M,"[B-P-Info, B-P-EC]",B-P-Info&EC,3,3,[merger-MergeConcat],[{}],"[{'nn_type_nn_name': 'merger-MergeConcat', 'Ba..."
3,R,[B-P-Info&EC],B-P,3,2,[reducer-ReduceMax],[{}],"[{'nn_type_nn_name': 'reducer-ReduceMax', 'Bas..."


In [24]:
df_SubUnit['SubUnit_BasicNN_Config_List'].iloc[2]

[{'nn_type_nn_name': 'merger-MergeConcat',
  'Basic_Config': {'input_names_nnlvl': ['B-P-Info', 'B-P-EC'],
   'output_name_nnlvl': 'B-P-Info&EC',
   'merger_para': {'nn_type': 'merger',
    'nn_name': 'MergeConcat',
    'nn_para': {},
    'input_size': 128,
    'output_size': 128,
    'postprocess': {'activator': 'gelu',
     'dropout': {'p': 0.5, 'inplace': False},
     'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}}}}]

# SubUnit

## Module

In [25]:
from fieldnn.module.subunit import SubUnit_Layer

## Usage

In [26]:
OutputTensor_2_Repr = {}

for idx, SubUnit_info in df_SubUnit.iterrows():
    # print(SubUnit_info)
    
    output_name = SubUnit_info['output_name']
    SubUnitLayer = SubUnit_Layer(SubUnit_info)
    OutputTensor_2_Repr[output_name] = SubUnitLayer
    

In [27]:
[i for i in RECFLD_TO_EMBEDTESNOR]

['B-P-Info', 'B-P-EC-PNSect-Tknz']

In [28]:
# fld_updates_dict = {}
# for i in RECFLD_TO_EMBEDTESNOR:
#     layernum = len(i.split('-'))
#     fld = i.split('-')[-1]
#     if '@' in fld:
#         # print(fld)
#         neat_i = '-'.join(i.split('-')[:-1]) + '-' + fld.split('@')[0]
#         # print(neat_i)
#         same_neat_list = [t for t in RECFLD_TO_EMBEDTESNOR if neat_i + '@' in t]
#         # print(same_neat_list)
#         if len(same_neat_list) == 1: # itself
#             # RECFLD_TO_EMBEDTESNOR[]
#             fld_updates_dict[i] = neat_i
            
# for old, new in fld_updates_dict.items():
#     RECFLD_TO_EMBEDTESNOR[new] = RECFLD_TO_EMBEDTESNOR.pop(old)

In [29]:
[i for i in RECFLD_TO_EMBEDTESNOR]

['B-P-Info', 'B-P-EC-PNSect-Tknz']

In [30]:
OUTPUT_TO_TENSOR = RECFLD_TO_EMBEDTESNOR.copy()

for output_name, SubUnitLayer in OutputTensor_2_Repr.items():
    input_names = SubUnitLayer.SubUnit_input_names
    SubUnit_output_name, info_dict = SubUnitLayer(input_names, OUTPUT_TO_TENSOR)
    
    print(SubUnit_output_name)
    OUTPUT_TO_TENSOR[SubUnit_output_name] = info_dict

B-P-EC-PNSect
B-P-EC
B-P-Info&EC
B-P


In [31]:
info_dict = OUTPUT_TO_TENSOR['B-P']

info_dict['info'].shape

torch.Size([4, 128])

# ReprBlock

## Module

In [32]:
import torch
# from .subunit import SubUnit_Layer


class ReprBlockLayer(torch.nn.Module):
    
    def __init__(self, df_SubUnit):
        super(ReprBlockLayer, self).__init__()
        self.df_SubUnit = df_SubUnit
        self.SubUnitDict = torch.nn.ModuleDict()
        
        for idx, SubUnit_info in df_SubUnit.iterrows():
            output_name = SubUnit_info['output_name']
            SubUnitLayer = SubUnit_Layer(SubUnit_info)
            self.SubUnitDict[output_name] = SubUnitLayer

    def forward(self, RECFLD_TO_EMBEDTESNOR):
        
        OUTPUT_TO_TENSOR = RECFLD_TO_EMBEDTESNOR.copy()
        
        for output_name, SubUnitLayer in self.SubUnitDict.items():
            input_names = SubUnitLayer.SubUnit_input_names
            SubUnit_output_name, info_dict = SubUnitLayer(input_names, OUTPUT_TO_TENSOR)
            
            assert output_name == SubUnit_output_name
            OUTPUT_TO_TENSOR[SubUnit_output_name] = info_dict
        
        return OUTPUT_TO_TENSOR
    

## Usage

In [33]:
ReprBlock = ReprBlockLayer(df_SubUnit)

# ReprBlock

In [34]:
# fld_updates_dict = {}
# for i in RECFLD_TO_EMBEDTESNOR:
#     layernum = len(i.split('-'))
#     fld = i.split('-')[-1]
#     if '@' in fld:
#         # print(fld)
#         neat_i = '-'.join(i.split('-')[:-1]) + '-' + fld.split('@')[0]
#         # print(neat_i)
#         same_neat_list = [t for t in RECFLD_TO_EMBEDTESNOR if neat_i + '@' in t]
#         # print(same_neat_list)
#         if len(same_neat_list) == 1: # itself
#             # RECFLD_TO_EMBEDTESNOR[]
#             fld_updates_dict[i] = neat_i
            
# for old, new in fld_updates_dict.items():
#     RECFLD_TO_EMBEDTESNOR[new] = RECFLD_TO_EMBEDTESNOR.pop(old)

In [35]:
[i for i in RECFLD_TO_EMBEDTESNOR]

['B-P-Info', 'B-P-EC-PNSect-Tknz']

In [36]:
OUTPUT_TO_TENSOR = ReprBlock(RECFLD_TO_EMBEDTESNOR)
[i for i in OUTPUT_TO_TENSOR]

['B-P-Info',
 'B-P-EC-PNSect-Tknz',
 'B-P-EC-PNSect',
 'B-P-EC',
 'B-P-Info&EC',
 'B-P']

In [48]:
holder = OUTPUT_TO_TENSOR['B-P']['holder']#.shape
info   = OUTPUT_TO_TENSOR['B-P']['info']#.shape

In [49]:
info.shape

torch.Size([4, 128])

In [50]:
holder

tensor([50, 66, 44, 65])